In [4]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv("/home/darshan39/Downloads/100_Unique_QA_Dataset.csv")

In [6]:
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [31]:
#tokenize
def tokenize(text):
    text=text.replace("?", " ")
    text = text.replace("'", " ")
    return text.split()

In [32]:
tokenize("What is the capital of France?")

['What', 'is', 'the', 'capital', 'of', 'France']

In [33]:
#vocab
vocab = {"<UNK>": 0}
type(vocab)

dict

In [34]:
def build_vocab(row):
    tokenized_question = tokenize(row['question'])
    tokenized_answer = tokenize(row['answer'])
    print(tokenized_question, tokenized_answer)